<a href="https://colab.research.google.com/github/tuyishimejohnson/chatbot/blob/main/agricultural_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00


In [2]:
# Import the necessary libaries
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict


In [3]:
# Load and check the structure of a dataset
ds = load_dataset("KisanVaani/agriculture-qa-english-only")

print(ds["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22615 [00:00<?, ? examples/s]

{'question': 'why is crop rotation important in farming?', 'answers': 'This helps to prevent soil erosion and depletion, and can also help to control pests and diseases'}


In [4]:
# Convert the dataset into panda dataframe for easy preprocessing
df = pd.DataFrame(ds["train"])

In [5]:
print(df.head())

                                            question  \
0         why is crop rotation important in farming?   
1  What farming practice helps prevent soil erosion?   
2                              what is crop rotation   
3      what are the different methods of irrigation?   
4                          why is soil health vital?   

                                             answers  
0  This helps to prevent soil erosion and depleti...  
1                                      Crop Rotation  
2  Crop rotation is the practice of growing a ser...  
3  surface irrigation, drip irrigation, and sprin...  
4  Soil health is critical to crop growth and pro...  


In [6]:
# Check for the missing values
print(df.isnull().sum())

question    0
answers     0
dtype: int64


In [7]:
# Double check to drop the missing values even though there are not present
df.dropna(inplace=True)



In [8]:
# Remove special characters, extraspaces and lowercase all letters
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9?.!,']", " ", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df['question'] = df['question'].apply(clean_text)
df['answer'] = df['answers'].apply(clean_text)


In [9]:
# Tokenize the model using wordPiece tokenization for BERT

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answers"], padding="max_length", truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/22615 [00:00<?, ? examples/s]

In [10]:
# Get the tokenized samples

print(tokenized_ds)
print(tokenized_ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22615
    })
})
{'question': 'why is crop rotation important in farming?', 'answers': 'This helps to prevent soil erosion and depletion, and can also help to control pests and diseases', 'input_ids': [101, 2339, 2003, 10416, 9963, 2590, 1999, 7876, 1029, 102, 2023, 7126, 2000, 4652, 5800, 14173, 1998, 2139, 10814, 3508, 1010, 1998, 2064, 2036, 2393, 2000, 2491, 20739, 2015, 1998, 7870, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
# Prepare data for training by importing TFAutoModelForSequenceClassification for question answering

''' from transformers import TFAutoModelForSequenceClassification

# Load the model (BERT-base)
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) '''



from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Convert dataset to TensorFlow format

train_features = {key: tf.constant(tokenized_ds["train"][key]) for key in tokenizer.model_input_names}
train_labels = tf.constant([0] * len(tokenized_ds["train"]["question"]))  # Dummy labels for now

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.batch(8).shuffle(1000)


In [13]:
from transformers import AdamWeightDecay

# Set optimizer and loss
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


# Training the model with few epochs
model.fit(train_dataset, epochs=1)

2827/2827 [==============================] - 2770s 972ms/step - loss: 0.0135 - accuracy: 0.9992


In [14]:
# Save the model and tokenizer
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [17]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r saved_model /content/drive/MyDrive/chatbot

Mounted at /content/drive


In [15]:
# Load the saved model and tokenizer later
loaded_model = TFAutoModelForQuestionAnswering.from_pretrained("saved_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("saved_model")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [16]:
# A function to display the response

def chatbot_response(user_input, context="Agriculture-related information goes here"):
    # Prepare the input
    input_text = f"{context}\n\nUser: {user_input}\n\nChatbot:"
    inputs = loaded_tokenizer(input_text, return_tensors="tf")

    # Generate a response
    output_sequences = loaded_model.generate(
        input_ids=inputs["input_ids"],
        max_length=100,  # Adjust as needed
        temperature=0.7,  # Control randomness
        top_k=50,
        top_p=0.9,
        do_sample=True  # Enables diverse responses
    )

    # Decode the generated text
    response = loaded_tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Extract only the chatbot's response (optional, to clean up output)
    response = response.split("Chatbot:")[-1].strip()

    return response

# Example Usage
print(chatbot_response("What are the best crops for dry seasons?"))

TypeError: The current model class (TFBertForQuestionAnswering) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'TFBertLMHeadModel'}

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))
